# Symbolic Expressions

## Contents

+ Modeling algebraic expressions as data structures.
+ Writing code to analyze, transform, or evaluate algebraic expressions.
+ Finding the derivative of a function by manipulating the expression that defines it.
+ Writing a Python function to compute derivative formulas.
+ Using [SymPy](https://www.sympy.org/en/index.html) to compute integral formulas.

## Intro

The previous two chapters introduces two of the most important concepts in calculus: the derivative and the integral.

We saw that you can approximate the derivative of a function at a point by taking slopes of smaller and smaller secant lines. We also saw that you can approximate the integral of a function by estimating the area under the graph with skinnier and skinnier rectangles.

Up until now we've worked with approximations, because we were relying on the capacity of Python running on a computer, but Calculus teaches us how to obtain exact values for the derivative and integral.

For example, if $ f(x) = x^3 $, Calculus tells us that:

$
f'(x) = 3 \cdot x^2
$

There are formulas for each type of function, that you would apply when asked for the exact derivative of a function.

However, as developers, knowing the formulas is not a very interesting skill for us. Instead, we can realy on a specialized tool called *computer algebra system* to compute it for us.

## Finding an exact derivative with a computer algebra system

Mathematica is one of the most popular computer algebra systems. Its engine is free only at [Wolfram Alpha](https://www.wolframalpha.com/).

This site can be used to find the exact formula for a derivative.

For example, when building neural networks, it's useful to know the derivative of the function:

$
\displaystyle
f(x) = \frac{1}{1 + e^{-x}}
$

This can be easily obtained by entering that formula in Python syntax. After pressing enter, it will display several facts about the function, including its derivative:

![Entering formula](pics/wolfram-alpha-input-formula.png)

![Derivative and Integral](pics/wolfram-alpha-derivative-integral.png)

Mathematica does not rely on the approximations we've worked on the previous chapters. Instead, Wolfram Alpha interprets the formula you've typed in, transforms it with some algebraic manipulations, and outputs a new formula.

This is called **symbolic programming**.

### Doing symbolic algebra in Python

Let's start by illustrating how we will be representing and manipulating formulas in Python.

Let's assume we have a mathematical function like:

$
f(x) = (3 x^2  + x) \cdot sin(x)
$

We already know that the best way to represent the formula in order to evaluate it would be:

```python
from math import sin

def f(x):
  return (2 * x ** 2 + x) * sin(x)
```

But that representation is not helpful to learn facts about the formula, such as:

+ Does the formula depend on the variable x?
+ Does it contain a trigonometric function?
+ Does it involve the operation of division?

While we can really easily answer those questions just by looking at the function, the Python representation used to get the function values wouldn't help.

For example, it'd be very difficult to write a function `contains_division(f)` that returns true if it uses the operation division in its definition.

Thus, we need to find another way to represent the function. We need another way to express the function that would tell us what operations are being applied and in what order.

For instance, the function $ f(x) $ above is a product of $ sin(x) $ with a sum. We also know that there's a well-known algebraic process for expanding the function definition:

$
f(x) = (3 x^2  + x) \cdot sin(x) = 3 x^2 \cdot sin(x) + x \cdot sin(x)
$

The new way to represent the formula should let us apply those rules as well.

In summary, we need to *model algebraic expressions as data structures* so that we can manipulate them symbolically, and therefore automate the rules of calculus.

Once we have that, we will be able to calculate derivatives.

Most functions expressed by simple formulas also have simple formulas for their derivatives. For example, if $ f(x) = x^3 $, $ f'(x) = 3 \cdot x^2 $.

That means that the derivative of $ x^3 $ at any value of $ x $ is $ 3 \cdot x^2 $.

Another requirement for our data structures is that we will have to be able to calculate the derivative or any given algebraic expression.

At the minimum, we will need to represent variables, numbers, sums, differences, products, quotients, powers, and functions such as sine and cosine and take the derivatives of them.

## Modeling algebraic expressions

Let's develop our intuition around how we can break an algebraic expression such as $ f(x) = (3 \cdot x^2 + x) \cdot sin(x) $.

It contains the following building blocks we should model:
+ a variable $ x $
+ numbers ($ 3 $)
+ operations: addition, multiplication, power
+ a named function: $ sin(x) $

The goal is to translate it into a Python data structure we can work with.

The first observation is that $ f $ is an arbitrary name. The right hand side will evaluate the same whether the function is called $ f $, $ g $, or $ h $.

As a result, we should be focusing on the right hand side of the function definition, which is called an expression:
> An expression is a collection of mathematical symbols combined in some valid ways.

### Breaking an expression into pieces

The way in which we can model algebraic expressions such as $ (3 \cdot x^2 + x) \cdot sin(x) $ is to break them into smaller expressions.

There is only one meaningful way to break up the expressions $ (3  \cdot x^2 + x) \cdot sin(x) $. Namely, it's the product of $ (3 \cdot x^2 + x) $ and $ sin(x) $.

Other ways of breaking up the expression will end up in something we can't make sense of according to mathematical rules:

![Breaking up expressions](pics/breaking_expressions.png)

If we apply the same strategy to the expression $ 3 \cdot x^2 + x $ we will see that it can be broken down as:

![expression tree](pics/expression_tree.png)

If we inspect the approach we see that we take:
+ operations &mdash; ways to take two or more algebraic expressions and stick them together side by side to make a new, bigger algebraic expression.
+ operators &mdash; valid places to break up an existing algebraic expression into smaller ones.


In the terminology of functional programming, functions combining smaller objects into bigger ones like this are often called *combinators*.

Some of the combinators we've used in the expression above are:
+ $ 3 \cdot x^2 $ is the product of the expression 3 and $ x^2 $
+ $ x^2 $ is a power: one expression $ x $ raise to the power of another expression $ 2 $.
+ $ sin(x) $ is a *function application*. Given the expression $ sin $ and the expression $ x $, we can build a new expression $ sin(x) $.

By contrast, a variable $ x $, a number $ 2 $, or a function named $ sin $ cannot be broken down further, and we call them *elements*.

### Building an expression tree

Let's focus on the expression $ (3 \cdot x^2 + x) \cdot sin(x) $. The elements $ 3 $, $ x $, $ 2 $, and $ sin $, along with the combinators $ adding $, $ multiplying $, $ \text{raising to a power} $, and $ \text{applying a function} $ are sufficient to rebuild the whole expression.

One of the first sub-expressions we can put together is $ x^2 $, which combines $ x $ and $ 2 $ with the $ power $ combinator. We can picture it as below:

![Subexpression 1](pics/subexpression_1.png)

Note how the subexpression is represented as a tree with the combinator at the root, and leaves $ x $ and $ 2 $.

A good next step would be to represent $ 3 \cdot x^2 $, which we could also represent visually as below:

![Subexpression 2](pics/subexpression_2.png)

Note how it still has the aspect of a tree, and that the left leave of the $ Product $ combinator is itself the previous tree that represented $ x^2 $.

We can add one more layer to the tree to represent the $ 3 \cdot x^2 + x $ expression:

![Subexpression 3](pics/subexpression_3.png)

Finally, we can represent the whole $ (3 \cdot x^2 + x) \cdot sin(x) $ by introducing the $ Apply $ combinator and the known $ Product $ combinator:

![Subexpression 4](pics/subexpression_4.png)

![Final expression](pics/expression.png)

Note how the tree reveals the structure of the algebraic expression. The root of the tree is the $ Product $ combinator, with two branches coming out of it: $ Sum $ and $ Apply $. 

Each combinator appearing further down the tree adds additional branches until you reach the elements that are leaves and have no branches. And all we have used are numbers, variables, named functions and operations (as combinators).

### Translating the expression tree to Python

We can use Python classes to represent each kind of element and each kind of combinator. As we go along, we will be revising their implementation to give them more and more functionality.

The idea is to model combinators as containers that hold their required inputs. For example, to model the $ Power $ combinator, we can do:

```python
class Power():
    def __init__(self, base, exponent):
        self.base = base
        self.exponent = exponent
```

That would allow us to write `Power("x", 2)`, but because we're dealing with symbolic expressions, it's even better to define special container classes for variables and numbers:

```python
class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol
```

The will let us represent $ x^2 $ in Python as `Power(Variable("x"), Number(2))`.

It is easy now to extend the previous approach to represent the $ Product $ operation:

```python
class Product():
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2
```

So that $ 3 \cdot x^2 $ can now be written as:

```python
Product(Number(3), Power(Variable("x"), Number(2)))
```

We can now define the rest of combinators:

```python
class Sum():
    def __init__(self, *exprs):
        self.exprs = exprs

class Function():
    def __init__(self, name):
        self.name = name

class Apply()
    def __init__(self, function, argument):
        self.function = function
        self.argument = argument
```

With those elements in place, we can faithfully represent the expression $ (3 \cdot x^2 + x) \cdot sin(x) $:

```python
expr = Product(
    Sum(
        Product(
            Number(3),
            Power(Variable("x"), Number(2))
        ),
        Variable("x")
    ),
    Apply(
        Function("sin"),
        Variable("x")
    )
)
```

By faithfully, we mean that we can have a look at the Python object and understand that it described the algebraic expression $ (3 \cdot x^2 + x) \cdot sin(x) $ and no other.

We started from an algebraic expression and built a Python object that uniquely and unequivocally.

We can also follow the opposite journey: start from a Python object and get to the algebraic expression it represents:

```python
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")), Number(−5)))
```

This represents the expression:

$
cos(x^3 - 5)
$

### Exercise

The natural algorithm, a special mathematical function, is written as $ ln(x) $.
Draw the expression $ ln(yz) $ as an expression tree built from the elements and combinators introduced in the previous section.



The tree can be displayed as:

![ln(yz) expr](pics/expr_lnyz.png)

The equivalent Python object will be:

```python
Apply(
    "ln",
    Product(
        Variable("y"),
        Variable("z")
    )
)
```


### Exercise

Consider the mathematical expression $ ln(y^z) $. Draw the expression tree.

Then, translate the expression into Python code, given that the natural algorithm is calculated by the Python function `math.log`. Write it as a Python object using the classes defined in the previous exercise.

The expression tree is the following:

![ln(y^z)](pics/expr_ln_y_power_z.png)

The Python function can be defined as follows:

In [2]:
import math

def fn(y, z):
    return math.log(y ** z)

print(fn(2, 3))
print(math.log(8))

2.0794415416798357
2.0794415416798357


And the expression tree using the classes defined above is:

```python
Apply(
    Function("ln"),
    Power(
        Variable("y"),
        Variable("z")
    )
)
```

### Exercise

What is the expression represented by `Product(Number(3), Sum(Variable("y"), Variable("z")))`?

The Python expression above is equivalent to the mathematical expression:

$
3 \cdot (y + z)
$

Note that the parentheses are not required in the Python expression but needed in the equivalent mathematical expression because $ \cdot $ has greater precedence than $ + $.

### Exercise

Implement a `Quotient` combination representing one expression divided by another. How do you represent the following expression?

$$
\frac{a + b}{2}
$$

We can follow the same approach we used for other binary combinators:

```python
class Quotient():
    def __init__(self, numerator_expr, denominator_expr):
        self.numerator_expr = numerator_expr
        self.denominator_expr = denominator_expr
```

Then we can write:

```python
Quotient(
    Sum(
        Variable("a"),
        Variable("b")
    ),
    Number(2)
)
```

### Exercise

Implement a `Difference` combinator representing one expression subtracted from another. How can you represent the expression $ b^2 - 4ac $?

We can follow the same approach used for the other binary combinators:

```python
class Difference:
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2
```

Then, the given expression can be written as:

```python
Difference(
    Power(
        Variable("b"),
        Number(2)
    ),
    Product(
        Number(4),
        Product(
            Variable("a"),
            Variable("c")
        )
    )
)
```

### Exercise

Implement a `Negative` combinator representing the negation of an expression. For example, the negation of $ x^2 + y $ is $ -(x^2 +  y) $. Represent the latter expression using your new combinator.

This time we are dealing with a *unary* combinator, but the approach is similar:

```python
class Negative:
    def __init__(self, expr):
        self.expr = expr
```

And the representation will be:

```python
Negative(
    Sum(
        Power(
            Variable("x"),
            Number(2)
        ),
        Variable("y")
    )
)
```

### Exercise

Add a function called `Sqrt` that represents a square root and use it to encode the following formula:

$$
\frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
$$

We can define the `Sqrt` function in the same way we define the `sin`:

```python
Sqrt = Function("sqrt")
```

Then the expression can be defined as:

```python
Quotient(
    Sum(
        Negative(Variable("b")),
        Apply(
            Sqrt,
            Difference(
                Power(
                    Variable("b"),
                    Number(2)
                ),
                Product(
                    Number(4),
                    Product(
                        Variable("a"),
                        Variable("c")
                    )
                )
            )
        )
    ),
    Product(
        Number(2),
        Variable("a")
    )
)
```

### Exercise